# Assignment 1: Operating Room Scheduling

## Question A

Python Code:

In [3]:
# Import gurobi
from gurobipy import *
import numpy as np

# Define model and parameters. 
mod = Model()

n_i = 6 # departments
n_j = 5 # operating rooms
n_k = 5 # days

d = np.array([[9, 9, 9, 9, 9], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [7.5, 7.5, 7.5, 7.5, 6.5]])
t = np.array([213.5 * 0.484, 213.5 * 0.042, 213.5 * 0.253, 213.5 * 0.074, 213.5 * 0.053, 213.5 * 0.095])

# Define decision variables.
x = mod.addVars(n_i, n_j, n_k, vtype = GRB.BINARY)
s = mod.addVars(n_i)

# Constraint 1 - each operating room time has one department at most
or_con = {}
for j in range(n_j):
    for k in range(n_k):
        or_con[j, k] = mod.addConstr(sum(x[i, j, k] for i in range (n_i)) <= 1)

# Constraint 2 - penalty for each department
under_con = {}
for i in range(n_i):
    under_con[i] = mod.addConstr(s[i] >= t[i] - sum(x[i, j, k] * d[j, k] for j in range(n_j) for k in range(n_k)))

# Constraint 3 - making 's' an under-allocation only
s_con = {}
for i in range(n_i):
    s_con[i] = mod.addConstr(s[i] >= 0)

# Create the objective function, and set it to be minimized.
mod.setObjective(sum(s[i] / t[i] for i in range(n_i)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 37 rows, 156 columns and 312 nonzeros
Model fingerprint: 0xa110901a
Variable types: 6 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 2.066116e-03, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207 

## Question B

Python Code:

In [5]:
# Define model and parameters. 
mod = Model()

n_i = 6 # departments
n_j = 5 # operating rooms
n_k = 5 # days
n_l = 3 # levels

d = np.array([[9, 9, 9, 9, 9], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [9, 9, 9, 9, 8], [7.5, 7.5, 7.5, 7.5, 6.5]])
t = np.array([213.5 * 0.484, 213.5 * 0.042, 213.5 * 0.253, 213.5 * 0.074, 213.5 * 0.053, 213.5 * 0.095])

# Define decision variables.
x = mod.addVars(n_i, n_j, n_k, vtype = GRB.BINARY)
s = mod.addVars(n_i)
v = mod.addVars(n_i, n_k, n_l)

# Constraint 1 - each operating room time has one department at most
or_con = {}
for j in range(n_j):
    for k in range(n_k):
        or_con[j, k] = mod.addConstr(sum(x[i, j, k] for i in range (n_i)) <= 1)

# Constraint 2 - penalty for each department
under_con = {}
for i in range(n_i):
    under_con[i] = mod.addConstr(s[i] >= t[i] - sum(x[i, j, k] * d[j, k] for j in range(n_j) for k in range(n_k)))

# Constraint 3 - making 's' an under-allocation only
s_con = {}
for i in range(n_i):
    s_con[i] = mod.addConstr(s[i] >= 0)
    
# Constraint 4 - one department must be on the same floor on same day
level_con = {}
for i in range(n_i):
    for k in range(n_k):
        level_con[i, k] = mod.addConstr(sum(v[i, k, l] for l in range(n_l)) <= 1)

# Constraint - determining which rooms are on each floor
level_1_con = {}
level_2_con = {}
level_3_con = {}
for i in range (n_i):
    for k in range(n_k):
        level_1_con[i, k] = mod.addConstr(sum(x[i, j, k] for j in [0, 1]) <= 2 * v[i, k, 0])
        level_2_con[i, k] = mod.addConstr(sum(x[i, j, k] for j in [2, 3]) <= 2 * v[i, k, 1])
        level_3_con[i, k] = mod.addConstr(x[i, 4, k] == v[i, k, 2])
        

# Create the objective function, and set it to be minimized.
mod.setObjective(sum(s[i] / t[i] for i in range(n_i)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 157 rows, 246 columns and 642 nonzeros
Model fingerprint: 0x4cbd3822
Variable types: 96 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 96 rows and 90 columns
Presolve time: 0.00s
Presolved: 61 rows, 156 columns, 456 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)
Found heuristic solution: objective 5.9998574

Root relaxation: objective 1.387152e-01, 57 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.13